# LRE on Hardware

This is a notebook for testing LRE on IBM Hardware

## Executor and Backend Setup

First import all required libraries and modules.

In [6]:
import json
import pandas as pd
import os
from datetime import date
import time
import matplotlib.pyplot as plt
import numpy as np
from typing import List

from functools import partial
import qiskit
from qiskit_aer import QasmSimulator
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler, QiskitRuntimeService
from qiskit_aer.noise import NoiseModel

import cache_utils

from mitiq import lre, zne, Executor
from mitiq.interface.mitiq_qiskit.qiskit_utils import initialized_depolarizing_noise
from mitiq.lre.multivariate_scaling.layerwise_folding import _get_num_layers_without_measurements

Define an executor that processes circuits serially.

In [7]:
def ibmq_executor(circuit: qiskit.QuantumCircuit, backend, shots: int = 1000) -> float:
    """Returns the expectation value to be mitigated.

    Args:
        circuit: Circuit to run (assumes no measurements).
        shots: Number of times to execute the circuit to compute the expectation value.
    """
    # Transpile the circuit so it can be properly run
    exec_circuit = qiskit.transpile(
        circuit,
        backend=backend,
        basis_gates=None,
        optimization_level=0,  # Important to preserve folded gates.
    )
    exec_circuit.measure_active()

    num_qubits = circuit.num_qubits

    # Run the circuit
    sampler = Sampler(mode=backend)
    job = sampler.run([exec_circuit], shots=shots)

    # Convert from raw measurement counts to the expectation value
    counts = job.result()[0].data.measure.get_counts()
    ground_state = "0" * num_qubits
    expectation_value = counts.get(ground_state, 0.0) / shots
    return expectation_value

Define a batch executor.

In [8]:
def batched_ibmq_executor(circuits: List[qiskit.QuantumCircuit], backend, shots: int = 1000) -> List[float]:
    """Returns the expectation value to be mitigated.

    Args:
        circuit: Circuit to run (assumes no measurements).
        shots: Number of times to execute the circuit to compute the expectation value.
    """

    num_qubits = len(circuits[0].qubits)

    final_circuits = []
    final_exp_values = []

    # Transpile the circuit so it can be properly run
    for circuit in circuits:
        exec_circuit = qiskit.transpile(
            circuit,
            backend=backend,
            basis_gates=None,
            optimization_level=0,  # Important to preserve folded gates.
        )
        exec_circuit.measure_active()
        final_circuits.append(exec_circuit)

    # Run the circuit
    sampler = Sampler(mode=backend)
    job = sampler.run(final_circuits, shots=shots)

    # Convert from raw measurement counts to the expectation value
    for result in job.result():
        counts = result.data.measure.get_counts()
        ground_state = "0" * num_qubits
        expectation_value = counts.get(ground_state, 0.0) / shots
        final_exp_values.append(expectation_value)
    return final_exp_values

In [9]:
#################################################
# HARDWARE
#################################################
service = QiskitRuntimeService()
hard_backend = service.least_busy(operational=True, simulator=False)
# hard_backend = service.backend("ibm_sherbrooke")
print(hard_backend)
hardware_executor = partial(ibmq_executor, backend=hard_backend)


#################################################
# SIMULATOR
#################################################
noise_model = NoiseModel.from_backend(hard_backend)
sim_backend = AerSimulator.from_backend(hard_backend)
simulator_executor = partial(ibmq_executor, backend=sim_backend)
partial_batched = partial(batched_ibmq_executor, backend=sim_backend)
batched_simulator_executor = Executor(partial_batched) # default max_batched_size= 75

# old_sim_backend = QasmSimulator(method="density_matrix", noise_model=noise_model)
# new_sim_backend = AerSimulator(method="density_matrix", noise_model=noise_model)

sim_backend_ideal = QasmSimulator()

<IBMBackend('ibm_sherbrooke')>


In [ ]:
batched_simulator_executor.can_batch

## Hyperparameter Tuning

First step is to do tests on simulators for hyperparamter tuning. With LRE, the primary paramters that can be adjusted are `degree` and `fold_multiplier`. Due to performance, it has been noted that `num_chunks` will also be critical for testing.

### Manual Example for testing

In [12]:
num_qubits = 4 # 4, 9, 16, 25, 36, 49 (available through benchpress)
# qasm_filename = f"square-heisenberg-{num_qubits}"
qasm_filename = f"qft-{num_qubits}"
circuit = qiskit.qasm2.load(f"{qasm_filename}.qasm")

CIRCUIT_NAME = qasm_filename
DEGREE = 3
FOLD_MULTIPLIER = 2
NUM_CHUNKS = 0

ideal = ibmq_executor(circuit, sim_backend_ideal)
unmitigated = ibmq_executor(circuit, sim_backend)
# new_unmitigated = ibmq_executor(circuit, new_sim_backend)
# old_unmitigated = ibmq_executor(circuit, old_sim_backend)



mitigated = lre.execute_with_lre(
    circuit, batched_simulator_executor, degree=10, fold_multiplier=10, num_chunks=2
)

# mitigated = zne.execute_with_zne(
#     circuit, batched_simulator_executor
# )

# print(f"ideal={ideal:.3f} unmitigated={unmitigated:.3f}")
print(f"ideal={ideal:.3f} unmitigated={unmitigated:.3f} mitigiated={mitigated:.3f}")

#################################################
# save ideal(time = 0.0) and unmitigated(time = 1.0)
#################################################
# ideal_res = [ qasm_filename, sim_backend.name, ideal, 0, 0, 0, 0]
# unmitigated_res = [ qasm_filename, sim_backend.name, unmitigated, 0, 0, 0, 1]
# cache_utils.save_result(ideal_res, f"results/{qasm_filename}_{date.today().strftime("%m-%d-%y")}_results.csv")
# cache_utils.save_result(unmitigated_res, f"results/{qasm_filename}_{date.today().strftime("%m-%d-%y")}_results.csv")


/Users/briangoldsmith/anaconda3/envs/mitiqMain/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


ValueError: Determinant of sample matrix cannot be calculated as the matrix is too large. Consider chunking your input circuit. 

In [ ]:
print(circuit)
print(len(circuit))

### Automated Example with Saving Results

First setup the parameters to test including: degree, fold_multiplier, and num_chunks.

In [4]:
MAX_DEGREE = 10
MAX_FOLD_MULTIPLIER = 5 
MAX_NUM_CHUNKS = 10 

# num_qubits = [4, 9, 16, 25, 36, 49]
num_qubits = [9]
qasm_filenames = [f"square-heisenberg-{num_qubit}" for num_qubit in num_qubits]
# qasm_filenames = [f"qft-{num_qubit}" for num_qubit in num_qubits]

# Setup all of the different values/circuits to test
circuits = [qiskit.qasm2.load(f"{qasm_filename}.qasm") for qasm_filename in qasm_filenames]
degrees = list(range(1,MAX_DEGREE+1))
fold_multipliers = list(range(1, MAX_FOLD_MULTIPLIER+1))
num_chunks = list(range(1, MAX_NUM_CHUNKS+1))


Run experiments on a noisy simulator to get results and store them into a results/cached_results.csv for later analysis.

In [ ]:
for i in range(len(circuits)):
    circuit = circuits[i]
    for fold_multiplier in fold_multipliers:
        for degree in degrees:
            for num_chunk in num_chunks:
                if num_chunk >= len(circuit):
                    # print(f"NUM_CHUNKS: fold={fold_multiplier} degree={degree} num_chunk={num_chunk} lenght={len(circuit)}")
                    break
                t0 = time.time()
                try:
                    res = lre.execute_with_lre(circuit, simulator_executor, degree=degree, fold_multiplier=fold_multiplier, num_chunks=num_chunk)
                except RuntimeWarning as w:
                    # print(f"RUNTIME degree={degree} fold_multiplier={fold_multiplier} num_chunks={num_chunks}: {w}")
                    break
                except ValueError as e:
                    # print(f"VALUEERROR: {e}")
                    if "Determinant of sample matrix cannot be calculated as the matrix is too large" in str(e):
                        t0 = time.time()-t0
                        cache_res = [
                            qasm_filenames[i],
                            sim_backend.name,
                            np.nan,
                            degree,
                            fold_multiplier,
                            num_chunk,
                            t0
                        ]
                        cache_utils.save_result(cache_res, f"results/{qasm_filenames[i]}_{date.today().strftime("%m-%d-%y")}_results.csv")

                        break  # Go to the next iteration of the loop
                    else:  # Re-raise if it's a different ValueError
                        raise  # Important: Don't silently ignore other ValueErrors!
                except Exception as e: #Catch other exceptions
                    # print(f"EXCEPTION {e}")
                    raise #Reraise the exception

                t0 = time.time()-t0
                cache_res = [
                   qasm_filenames[i],
                   sim_backend.name,
                   res,
                   degree,
                   fold_multiplier,
                   num_chunk,
                   t0
                ]
                # cache_utils.save_result(cache_res)
                cache_utils.save_result(cache_res, f"results/{qasm_filenames[i]}_{date.today().strftime("%m-%d-%y")}_results.csv")

In [9]:
# use actual hardware
# 

unmitigated = ibmq_executor(circuit, hard_backend)


In [10]:
cache_res = [
                   qasm_filenames[i],
                   hard_backend.name,
                   unmitigated,
                   0,
                   0,
                   0,
                   1.0 
                ]

cache_utils.save_result(unmitigated_res, f"results/{hard_backend.name}_{date.today().strftime("%m-%d-%y")}_results.csv")

In [ ]:
ibmq_executor

In [ ]:
cache_res = [
                   qasm_filenames[i],
                   hard_backend.name,
                   mitigated,
                   2,
                   2,
                   7,
                   1.0 
                ]

cache_utils.save_result(unmitigated_res, f"results/{hard_backend.name}_{date.today().strftime("%m-%d-%y")}_results.csv")

In [ ]:
# Read from CSV results
file_name = 'results/cached_results.csv'
df = pd.read_csv(file_name)
df.sort_values(by=["circuit, result"])
squareh_4 = df.loc[(df["circuit"] == "square-heisenberg-4") & (df["backend"] == "aer_simulator_from(ibm_sherbrooke)")]
degrees = squareh_4["degree"]
fold_multipliers = squareh_4["fold_multiplier"]
num_chunks = squareh_4["num_chunks"]
results = squareh_4["result"]

plt.figure(figsize=(8,6))
scatter = plt.scatter(degrees, fold_multipliers, c=num_chunks, cmap="viridis", s = (1-results)*100)

plt.xlabel("degree")
plt.ylabel("fold_multiplier")
plt.title("Hyperparameter tuning for LRE for square-heisenberg-4")
plt.colorbar(scatter, label="num_chunks")

plt.grid(True)
plt.tight_layout()
plt.show()